In [1]:
import feather
import numpy as np
from sklearn import metrics
from sklearn import model_selection
from sklearn import preprocessing
from datetime import datetime
from math import sqrt

import pandas as pd
import tensorflow as tf

In [2]:
train_df = feather.read_dataframe("../../datasets/kaggle/new-york-city-taxi-fare-prediction/train.feather")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54246832 entries, 0 to 54246831
Data columns (total 19 columns):
fare_amount           float32
pickup_datetime       datetime64[ns]
pickup_longitude      float32
pickup_latitude       float32
dropoff_longitude     float32
dropoff_latitude      float32
passenger_count       uint8
distance_miles        float32
is_pickup_JFK_new     bool
is_dropoff_JFK_new    bool
is_pickup_EWR_new     bool
is_dropoff_EWR_new    bool
is_pickup_LGA_new     bool
is_dropoff_LGA_new    bool
is_to_from_JFK_new    bool
distance_to_center    float32
hour                  int64
weekday               int64
year                  int64
dtypes: bool(7), datetime64[ns](1), float32(7), int64(3), uint8(1)
memory usage: 3.4 GB


In [4]:
train_df["hour_period"]=train_df["hour"] // 4
train_df.head()

,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,distance_miles,is_pickup_JFK_new,is_dropoff_JFK_new,is_pickup_EWR_new,is_dropoff_EWR_new,is_pickup_LGA_new,is_dropoff_LGA_new,is_to_from_JFK_new,distance_to_center,hour,weekday,year,hour_period
0,4.5,2009-06-15 17:26:21,-73.844315,40.721317,-73.841614,40.712276,1,0.000000,False,False,False,False,False,False,False,8.479724,17,0,2009,4
1,16.9,2010-01-05 16:52:16,-74.016045,40.711304,-73.979271,40.782005,1,5.244397,False,False,False,False,False,False,False,0.000000,16,1,2010,4
2,5.7,2011-08-18 00:35:00,-73.982735,40.761269,-73.991241,40.750561,2,0.000000,False,False,False,False,False,False,False,3.504552,0,3,2011,0
3,7.7,2012-04-21 04:30:42,-73.987129,40.733143,-73.991570,40.758091,1,1.932986,False,False,False,False,False,False,False,1.715006,4,5,2012,1
4,5.3,2010-03-09 07:51:00,-73.968094,40.768009,-73.956657,40.783764,1,1.366828,False,False,False,False,False,False,False,4.167420,7,1,2010,1


In [5]:
train_df = pd.get_dummies(train_df, prefix=["year","hour_period"], columns=["year","hour_period"])

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54246832 entries, 0 to 54246831
Data columns (total 31 columns):
fare_amount           float32
pickup_datetime       datetime64[ns]
pickup_longitude      float32
pickup_latitude       float32
dropoff_longitude     float32
dropoff_latitude      float32
passenger_count       uint8
distance_miles        float32
is_pickup_JFK_new     bool
is_dropoff_JFK_new    bool
is_pickup_EWR_new     bool
is_dropoff_EWR_new    bool
is_pickup_LGA_new     bool
is_dropoff_LGA_new    bool
is_to_from_JFK_new    bool
distance_to_center    float32
hour                  int64
weekday               int64
year_2009             uint8
year_2010             uint8
year_2011             uint8
year_2012             uint8
year_2013             uint8
year_2014             uint8
year_2015             uint8
hour_period_0         uint8
hour_period_1         uint8
hour_period_2         uint8
hour_period_3         uint8
hour_period_4         uint8
hour_period_5         uint8
d

In [7]:
def get_input_matrix(df):
    return np.column_stack((
        df.passenger_count,
        df.distance_miles,
        df.distance_to_center,
        df.is_to_from_JFK_new,
        df.year_2009,
        df.year_2010,
        df.year_2011,
        df.year_2012,
        df.year_2013,
        df.year_2014,
        df.year_2015,
        df.hour_period_0,
        df.hour_period_1,
        df.hour_period_2,
        df.hour_period_3,
        df.hour_period_4,
        df.hour_period_5,
        np.ones(len(df))))

In [8]:
train_X = get_input_matrix(train_df)
train_y = np.array(train_df['fare_amount'])

print(train_X.shape)
print(train_y.shape)

(54246832, 18)
(54246832,)


In [9]:
train_df = None

In [10]:
(w, _, _, _) = np.linalg.lstsq(train_X, train_y, rcond = None)
print(w)

[1.18553274 0.2160027  1.22984091 0.56364238 0.44505191 0.25326743
 0.82715883 0.32032715 3.03293032 0.98646442 3.57200497 0.33271525
 1.93275017 0.2479355  1.46926279 0.3065764  1.35157925 1.64113621]


In [11]:
from sklearn.metrics import mean_squared_error
from math import sqrt

In [12]:
train_y_w = np.matmul(train_X, w).round(decimals = 2)
rmse_w = sqrt(mean_squared_error(train_y, train_y_w))
print("rmse linalg.lstsq =", rmse_w)

rmse linalg.lstsq = 20.52402415205221


In [11]:
from sklearn.linear_model import Lasso
lr = Lasso(alpha=0.1)

In [12]:
lr.fit(train_X,train_y)

MemoryError: 

In [63]:
print(lr.coef_)
print(lr.intercept_)

[ 0.          3.14975037  0.09931013 -0.17242813 -0.07818841 -0.
 -0.          0.70425405  0.96325279  0.36606851 -0.          0.
  0.          0.          0.         -0.         -0.          0.        ]
4.894748762965194


In [64]:
y_pred = lr.predict(train_X)

In [65]:
print("rmse",sqrt(metrics.mean_squared_error(y_pred, train_y)))

rmse 19.238070451033153


In [67]:
print(y_pred.shape)

(54246832,)


In [66]:
test_df = feather.read_dataframe("../../datasets/kaggle/new-york-city-taxi-fare-prediction/test.feather")

In [68]:
test_df = pd.get_dummies(test_df, prefix=["year","weekday"], columns=["year","weekday"])

In [69]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9914 entries, 0 to 9913
Data columns (total 24 columns):
key                   9914 non-null object
pickup_datetime       9914 non-null datetime64[ns]
pickup_longitude      9914 non-null float32
pickup_latitude       9914 non-null float32
dropoff_longitude     9914 non-null float32
dropoff_latitude      9914 non-null float32
passenger_count       9914 non-null uint8
distance_miles        9914 non-null float32
distance_to_center    9914 non-null float32
hour                  9914 non-null int64
year_2009             9914 non-null uint8
year_2010             9914 non-null uint8
year_2011             9914 non-null uint8
year_2012             9914 non-null uint8
year_2013             9914 non-null uint8
year_2014             9914 non-null uint8
year_2015             9914 non-null uint8
weekday_0             9914 non-null uint8
weekday_1             9914 non-null uint8
weekday_2             9914 non-null uint8
weekday_3             9914 non-

In [70]:
test_X = get_input_matrix(test_df)
print(test_X.shape)

(9914, 18)


In [71]:
test_y_predictions = lr.predict(test_X)

# Write the predictions to a CSV file which we can submit to the competition.
submission = pd.DataFrame(
    {'key': test_df.key, 'fare_amount': test_y_predictions},
    columns = ['key', 'fare_amount'])
submission.to_csv('../../datasets/kaggle/new-york-city-taxi-fare-prediction/submission.csv', index = False)

In [72]:
submission.describe()

,fare_amount
count,9914.000000
mean,11.569597
std,8.323405
min,4.722321
25%,5.889315
50%,9.718892
75%,13.408325
max,206.886596
